In [1]:
import pandas as pd
import numpy as np
import re
import copy
import dtale
#pd.set_option("Max_columns", None)

### PITF 2018 data
- http://www.systemicpeace.org/inscrdata.html

In [3]:
# set different threshold of peace years prior to conflict
threshold_peace = 2
# year_lag
yr_p = threshold_peace
yr_l = 2

In [4]:
dt_pitf_rc = pd.read_excel("data/PITF 2018/PITF Adverse Regime Change 2018.xls")
dt_pitf_ew = pd.read_excel("data/PITF 2018/PITF Ethnic War 2018.xls")
dt_pitf_gp = pd.read_excel("data/PITF 2018/PITF GenoPoliticide 2018.xls")
dt_pitf_rw = pd.read_excel("data/PITF 2018/PITF Revolutionary War 2018.xls")
#
dt_pitf_rc_onging = dt_pitf_rc[["COUNTRY", "YEAR"]].copy()
dt_pitf_rc_onging.COUNTRY = dt_pitf_rc_onging.COUNTRY.str.strip()
dt_pitf_rc_onging["rc_ongoing"]= 1
dt_pitf_ew_onging = dt_pitf_ew[["COUNTRY", "YEAR"]].copy()
dt_pitf_ew_onging.COUNTRY = dt_pitf_ew_onging.COUNTRY.str.strip()
dt_pitf_ew_onging["ew_ongoing"]= 1
dt_pitf_gp_onging = dt_pitf_gp[["COUNTRY", "YEAR"]].copy()
dt_pitf_gp_onging.COUNTRY = dt_pitf_gp_onging.COUNTRY.str.strip()
dt_pitf_gp_onging["gp_ongoing"]= 1
dt_pitf_rw_onging = dt_pitf_rw[["COUNTRY", "YEAR"]].copy()
dt_pitf_rw_onging.COUNTRY = dt_pitf_rw_onging.COUNTRY.str.strip()
dt_pitf_rw_onging["rw_ongoing"]= 1
dt_pitf_consolidate = pd.merge(left=dt_pitf_rc_onging, right = dt_pitf_ew_onging, on=["COUNTRY", "YEAR"], how="outer")
dt_pitf_consolidate = pd.merge(left=dt_pitf_consolidate, right = dt_pitf_gp_onging, on=["COUNTRY", "YEAR"], how="outer")
dt_pitf_consolidate = pd.merge(left=dt_pitf_consolidate, right = dt_pitf_rw_onging, on=["COUNTRY", "YEAR"], how="outer")
dt_pitf_consolidate.fillna(0, inplace=True)
dt_pitf_consolidate.columns = [i.lower() for i in dt_pitf_consolidate.columns]
dt_pitf_consolidate.to_excel("data/PITF 2018/PITF Consolidate 2018.xlsx", index=False)

In [5]:
year_pitf = [y for y in range(dt_pitf_consolidate.year.min()-10, dt_pitf_consolidate.year.max()+1)]
c_list = []
y_list = []
for c in dt_pitf_consolidate.country.unique():
    for y in year_pitf:
        c_list.append(c)
        y_list.append(y)
dt_pitf_consolidate_long = pd.DataFrame({"country":c_list, "year":y_list})
dt_pitf_consolidate_long = pd.merge(left = dt_pitf_consolidate_long, right = dt_pitf_consolidate, on= ["country", "year"], how="left")
dt_pitf_consolidate_long.fillna(0, inplace=True)
dt_pitf_consolidate_long["conflict_ongoing"] = dt_pitf_consolidate_long[["rc_ongoing", "ew_ongoing", "gp_ongoing", "rw_ongoing"]].sum(axis=1)
for c in dt_pitf_consolidate_long.columns[2:]:
    dt_pitf_consolidate_long[c] = dt_pitf_consolidate_long[c].astype("int")
dt_pitf_consolidate_long.insert(2, "country_year", dt_pitf_consolidate_long.country +"_"+dt_pitf_consolidate_long.year.astype(str))

In [6]:
c_dt = dt_pitf_consolidate_long.country.sort_values().unique()
y_dt = dt_pitf_consolidate_long.year.sort_values().unique()
onset_pitf = []
onset_rconly_pitf = []
for c in c_dt:
    yrs_peace = 0
    for y in y_dt:
        conflict_ongoing = dt_pitf_consolidate_long[(dt_pitf_consolidate_long.country==c)&(dt_pitf_consolidate_long.year==y)].conflict_ongoing.values[0]
        rc_ongoing = dt_pitf_consolidate_long[(dt_pitf_consolidate_long.country==c)&(dt_pitf_consolidate_long.year==y)].rc_ongoing.values[0]
        if conflict_ongoing == 0:
            yrs_peace += 1
        else:
            if yrs_peace >= threshold_peace:
                onset_pitf.append(f"{c}_{y}")
                # if the onset case is regime change only, we will exclude
                if conflict_ongoing == rc_ongoing ==1:
                    onset_rconly_pitf.append(f"{c}_{y}")
            yrs_peace = 0        

### PRIO
- New DV from PRIO: 
    - 3 & 4 https://ucdp.uu.se/downloads/index.html#onesided  UCDP/PRIO Armed Conflict Dataset version 21.1
    - cumulative_intensity needs to be 1 for the span of a conflict
    - starting year is in start_date2
    - one unique conflict episode (classified by conflict_id) can have several events, i.e., different starting_date2

In [7]:
dt_prio = pd.read_csv("data/ucdp-prio-acd-211-csv/ucdp-prio-acd-211.csv")
dt_prio = dt_prio[(dt_prio.year<=2018)& (dt_prio.type_of_conflict.isin([3,4]))].reset_index(drop=True)
conflictid_cumu = []
for i in dt_prio.conflict_id.unique():
    if dt_prio[dt_prio.conflict_id==i].cumulative_intensity.sum()>=1:
        conflictid_cumu.append(i)
dt_prio = dt_prio[dt_prio.conflict_id.isin(conflictid_cumu)].reset_index(drop=True)

In [8]:
### concord to PITF data
def prio_concord(df):
    df.loc[df["location"]=="Bosnia-Herzegovina", "location"] = "Bosnia"
    df.loc[df["location"]=="Cambodia (Kampuchea)", "location"] = "Cambodia"
    df.loc[df["location"]=="Congo", "location"] = "Congo-Brazzaville"
    df.loc[df["location"]=="DR Congo (Zaire)", "location"] = "Congo-Kinshasa"
    df.loc[df["location"]=="Hyderabad", "location"] = "India"
    df.loc[df["location"]=="Madagascar (Malagasy)", "location"] = "Madagascar"
    df.loc[(df["location"]=="Russia (Soviet Union)") & (df["year"]<=1991), "location"] = "USSR"
    df.loc[(df["location"]=="Russia (Soviet Union)") & (df["year"]>1991), "location"] = "Russia"
    df.loc[df["location"]=="Serbia (Yugoslavia)", "location"] = "Yugoslavia"
    df.loc[df["location"]=="South Vietnam", "location"] = "Vietnam South"
    df.loc[df["location"]=="South Yemen", "location"] = "Yemen South"
    df.loc[(df["location"]=="Yemen (North Yemen)") & (df["year"]<=1989), "location"] = "Yemen North"
    df.loc[(df["location"]=="Yemen (North Yemen)") & (df["year"]>1989), "location"] = "Yemen"
    df.loc[df["location"]=="Zimbabwe (Rhodesia)", "location"] = "Zimbabwe"
    return df
# cases completly not covered in PITF
dt_prio = prio_concord(dt_prio)
for c in dt_prio.location.sort_values().unique():
    if c not in dt_pitf_consolidate.country.unique():
        print(c)

Bolivia
Cameroon
Costa Rica
Paraguay
United States of America


In [9]:
# threshold for reaching cumulative intensity
dt_prio_cumul = dt_prio[["conflict_id","location", "year", "cumulative_intensity", "start_date2"]].copy()
prio_ongoing = []
c_prio = []
y_prio = []
for c in dt_prio_cumul.location.sort_values().unique():
    dt_prio_c = dt_prio_cumul[dt_prio_cumul.location==c].copy()
    y_c_set = set()
    # conflict id
    for cid in dt_prio_c.conflict_id.unique():
        dt_prio_c_id = dt_prio_c[dt_prio_c.conflict_id==cid].copy()
        # start_date2
        for sd2 in dt_prio_c_id.start_date2.unique():
            dt_prio_c_id_sd2 = dt_prio_c_id[dt_prio_c_id.start_date2==sd2].copy()
            if dt_prio_c_id_sd2.cumulative_intensity.sum()>0:
                y_end = dt_prio_c_id_sd2.year.max()
                y_start = int(sd2[-4:])
                y_c_set = y_c_set | set(i for i in range(y_start, y_end+1))
    for y in y_c_set:
        c_prio.append(c)
        y_prio.append(y)
dt_prio_consolidate = pd.DataFrame({"country": c_prio , "year": y_prio})
dt_prio_consolidate["prio_ongoing"] = 1
dt_prio_consolidate = dt_prio_consolidate.sort_values(by=["country", "year"]).reset_index(drop=True)

In [10]:
year_prio = [y for y in range(dt_prio_consolidate.year.min()-10, dt_prio_consolidate.year.max()+1)]
c_list = []
y_list = []
for c in dt_prio_consolidate.country.unique():
    for y in year_prio:
        c_list.append(c)
        y_list.append(y)
dt_prio_consolidate_long = pd.DataFrame({"country":c_list, "year":y_list})
dt_prio_consolidate_long = pd.merge(left = dt_prio_consolidate_long, right = dt_prio_consolidate, on= ["country", "year"], how="left")
dt_prio_consolidate_long.fillna(0, inplace=True)
for c in dt_prio_consolidate_long.columns[2:]:
    dt_prio_consolidate_long[c] = dt_prio_consolidate_long[c].astype("int")
dt_prio_consolidate_long.insert(2, "country_year", dt_prio_consolidate_long.country +"_"+dt_prio_consolidate_long.year.astype(str))

In [11]:
# same threshold as PITF
c_dt = dt_prio_consolidate_long.country.sort_values().unique()
y_dt = dt_prio_consolidate_long.year.sort_values().unique()
onset_prio = []
for c in c_dt:
    yrs_peace = 0
    for y in y_dt:
        conflict_ongoing = dt_prio_consolidate_long[(dt_prio_consolidate_long.country==c)&(dt_prio_consolidate_long.year==y)].prio_ongoing.values[0]
        if conflict_ongoing == 0:
            yrs_peace += 1
        else:
            if yrs_peace >= threshold_peace:
                onset_prio.append(f"{c}_{y}")
            yrs_peace = 0        

In [12]:
dt_pitf_onset = dt_pitf_consolidate_long[dt_pitf_consolidate_long.country_year.isin([c for c in onset_pitf if c not in onset_rconly_pitf])].reset_index(drop=True)
dt_prio_onset = dt_prio_consolidate_long[dt_prio_consolidate_long.country_year.isin(onset_prio)].reset_index(drop=True)
#####
peace_time_list = []
for cy in dt_pitf_onset.country_year:
    c = cy.split("_")[0]
    y = int(cy.split("_")[1])
    dt_cy = dt_pitf_consolidate_long[(dt_pitf_consolidate_long.country==c)&(dt_pitf_consolidate_long.year<y) & (dt_pitf_consolidate_long.year>=1956)]
    dt_cy = dt_cy.sort_values(by=["year"], ascending=False).reset_index(drop=True)
    peace_time = 0
    for y in dt_cy.conflict_ongoing:
        if y>0:
            break
        else:
            peace_time+=1
    peace_time_list.append(peace_time)
dt_pitf_onset["peace_years"] = copy.deepcopy(peace_time_list)
#
peace_time_list = []
for cy in dt_prio_onset.country_year:
    c = cy.split("_")[0]
    y = int(cy.split("_")[1])
    dt_cy = dt_prio_consolidate_long[(dt_prio_consolidate_long.country==c)&(dt_prio_consolidate_long.year<y) & (dt_prio_consolidate_long.year>=1956)]
    dt_cy = dt_cy.sort_values(by=["year"], ascending=False).reset_index(drop=True)
    peace_time = 0
    for y in dt_cy.prio_ongoing:
        if y>0:
            break
        else:
            peace_time+=1
    peace_time_list.append(peace_time)
dt_prio_onset["peace_years"] = copy.deepcopy(peace_time_list)
####
dt_pitf_onset = dt_pitf_onset[["country", "year", "country_year", "conflict_ongoing", "peace_years"]].copy()
dt_pitf_onset.rename(columns={"conflict_ongoing": "PITF_onset"}, inplace=True)
dt_prio_onset.rename(columns={"prio_ongoing": "PRIO_onset"}, inplace=True)
####
onset_to_iv = {
    "Congo-Brazzaville": "Congo Brazzaville",
    "Congo-Kinshasa": "Congo Kinshasa",
    "South Sudan": "Sudan South",
    "Sudan-North": "Sudan",
    "United States of America": "United States",
}
dt_prio_onset.country = dt_prio_onset.country.replace(onset_to_iv)
dt_pitf_onset.country = dt_pitf_onset.country.replace(onset_to_iv)

In [13]:
dt_iv = pd.read_excel("data/Minerva_Clustering_20220209_MARupdated_Vdemadded.xlsx")
###
dt_pitf_onset = dt_pitf_onset.drop(columns=["country_year", "PITF_onset"])
dt_pitf_onset["year"] = dt_pitf_onset.year - yr_l
dt_pitf_onset = pd.merge(left=dt_iv, right=dt_pitf_onset, on =["country", "year"], how="right")
dt_pitf_onset["year"] = dt_pitf_onset.year + yr_l
dt_pitf_onset.insert(2, "country_year_onset", dt_pitf_onset.country + "_" + dt_pitf_onset.year.astype(str))
dt_pitf_onset = dt_pitf_onset.drop_duplicates()
###
dt_prio_onset = dt_prio_onset.drop(columns=["country_year", "PRIO_onset"])
dt_prio_onset["year"] = dt_prio_onset.year - yr_l
dt_prio_onset = pd.merge(left=dt_iv, right=dt_prio_onset, on =["country", "year"], how="right")
dt_prio_onset["year"] = dt_prio_onset.year + yr_l
dt_prio_onset.insert(2, "country_year_onset", dt_prio_onset.country + "_" + dt_prio_onset.year.astype(str))
dt_prio_onset = dt_prio_onset.drop_duplicates()
##
dt_pitf_onset["py_pt8"] = 2**(-dt_pitf_onset.peace_years/8)
dt_prio_onset["py_pt8"] = 2**(-dt_prio_onset.peace_years/8)
##
dt_pitf_onset.to_excel(f"data/PITF_{yr_p}yrpeace_{yr_l}yrlag.xlsx", index=False)
dt_prio_onset.to_excel(f"data/PRIO_{yr_p}yrpeace_{yr_l}yrlag.xlsx", index=False)